<a href="https://colab.research.google.com/github/nyp-sit/it3103/blob/main/week12/bert-finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/nyp-sit/sdaai-iti107/blob/master/session-8/bert-finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" align="left"/></a>

<br/>

# Fine-tuning BERT for Text Classification

One of the approach where we can use BERT for downstream task such as text classification is to do fine-tuning of the pretrained model. 

In this lab, we will see how we can use a pretrained DistilBert Model and fine-tune it with custom training data for text classification task. 

At the end of this session, you will be able to:
- prepare data and use model-specific Tokenizer to format data suitable for use by the model
- configure the transformer model for fine-tuning 
- train the model for binary and multi-class text classification


In [1]:
!pip install transformers

     |████████████████████████████████| 2.3MB 30.4MB/s 
     |████████████████████████████████| 3.3MB 45.2MB/s 
     |████████████████████████████████| 901kB 44.5MB/s 


In [31]:
import numpy as np
import tensorflow as tf
import pandas as pd

from transformers import (
    AutoConfig,
    AutoTokenizer,
    TFAutoModelForSequenceClassification,
    TFTrainer,
    TFTrainingArguments,
)
from transformers.utils import logging as hf_logging
from sklearn.model_selection import train_test_split

# We enable logging level to info and use default log handler and log formatting
hf_logging.set_verbosity_info()
hf_logging.enable_default_handler()
hf_logging.enable_explicit_format()

## Data Preparation

In [32]:
# Uncomment the following if you have not downloaded the datasets.

!wget https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/datasets/imdb_test.csv
!wget https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/datasets/imdb_train.csv

--2021-06-16 06:50:35--  https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/datasets/imdb_test.csv
Resolving nyp-aicourse.s3-ap-southeast-1.amazonaws.com (nyp-aicourse.s3-ap-southeast-1.amazonaws.com)... 52.219.32.143
Connecting to nyp-aicourse.s3-ap-southeast-1.amazonaws.com (nyp-aicourse.s3-ap-southeast-1.amazonaws.com)|52.219.32.143|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13308773 (13M) [text/csv]
Saving to: ‘imdb_test.csv.1’

imdb_test.csv.1     100%[===================>]  12.69M  4.87MB/s    in 2.6s    

2021-06-16 06:50:39 (4.87 MB/s) - ‘imdb_test.csv.1’ saved [13308773/13308773]

--2021-06-16 06:50:39--  https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/datasets/imdb_train.csv
Resolving nyp-aicourse.s3-ap-southeast-1.amazonaws.com (nyp-aicourse.s3-ap-southeast-1.amazonaws.com)... 52.219.36.31
Connecting to nyp-aicourse.s3-ap-southeast-1.amazonaws.com (nyp-aicourse.s3-ap-southeast-1.amazonaws.com)|52.219.36.31|:443... connected.
HTTP request

In [33]:
train_df = pd.read_csv('imdb_train.csv')
test_df = pd.read_csv('imdb_test.csv')

In [34]:
TRAIN_SIZE = 2500
TEST_SIZE = 200 

train_df = train_df.sample(n=TRAIN_SIZE)
test_df = test_df.sample(n=TEST_SIZE)

In [35]:
train_df['sentiment'] =  train_df['sentiment'].apply(lambda x: 0 if x == 'negative' else 1)
test_df['sentiment'] =  test_df['sentiment'].apply(lambda x: 0 if x == 'negative' else 1)

In [36]:
train_df.sentiment.value_counts()

0    1266
1    1234
Name: sentiment, dtype: int64

In [37]:
train_texts = train_df['review']
train_labels = train_df['sentiment']
test_texts = test_df['review']
test_labels = test_df['sentiment']

In [38]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

## Tokenization

We will now load the DistilBert tokenizer for the pretrained model "distillbert-base-cased".  The tokenizer helps to produce the input tokens that are suitable to be used by the model, e.g. it automatically append the \[CLS\] token in the front of the sentence and the \[SEP\] token at the end of the token, and also the attention mask for those padded positions in the input sequence of tokens.

In [39]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
#tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

[INFO|configuration_utils.py:517] 2021-06-16 06:50:46,073 >> loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
[INFO|configuration_utils.py:553] 2021-06-16 06:50:46,079 >> Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.6.1",
  "vocab_size": 30522
}

[INFO|tokenization_utils_base.py:1717] 2021-06-16 06:50:46,213 >> loading file https://hu

The DistilBERT tokenizer (identical to Bert tokenizer) use WordPiece vocabulary. It has close to 30000 words and it maps pretrained embeddings for each. Each word has its own ids, we would need to map the tokens to those ids.

In [40]:
print(f"Tokenizer vocab size = {tokenizer.vocab_size}")
print(list(tokenizer.vocab.keys())[6000:6020])

Tokenizer vocab size = 30522
['expectations', 'candace', 'fc', 'stitches', 'walks', '##β', 'affection', 'week', 'helmut', '##laid', '##lean', '##rard', 'ᆸ', '##shi', '##ica', 'continental', 'lovely', 'smith', 'orders', 'opponent']


Let us take a closer look at the output of the tokenization process. 

We notice that the tokenizer will return a dictionary of two items 'input_ids' and 'attention_mask'. The input_ids contains the IDs of the tokens. While the 'attention_mask' contains the masking pattern for those padding. If you are using BERT tokenizer, there will be additional item called 'token_type_ids'.

We also notice that for the example sentence, the word 'Transformer' is being broken up into two tokens 'Trans' and '##former'. Similarly, 'Processing' is tokenized as 'Process' and '##ing'.  The '##' means that the rest of the token should be attached to the previous one.

We also see that the tokenizer appended \[CLS\] to the beginning of the token sequence, and \[SEP\] at the end. 

In [41]:
test_sentence = "Transformer is really good for Natural Language Processing."

encoding = tokenizer(test_sentence, padding=True, truncation=True)
print(f"Encoding keys:  {encoding.keys()}\n")

print(f"token ids: {encoding['input_ids']}\n")

print(f"tokens: {tokenizer.convert_ids_to_tokens(encoding['input_ids'])}")


Encoding keys:  dict_keys(['input_ids', 'attention_mask'])

token ids: [101, 10938, 2121, 2003, 2428, 2204, 2005, 3019, 2653, 6364, 1012, 102]

tokens: ['[CLS]', 'transform', '##er', 'is', 'really', 'good', 'for', 'natural', 'language', 'processing', '.', '[SEP]']


Now let's go ahead and tokenize our texts. But before we do so, we need to convert the pandas series to list first as the tokenizer cannot work with pandas series or dataframe directly. 

In [42]:
train_texts = train_texts.to_list()
train_labels = train_labels.to_list()
val_texts = val_texts.to_list()
val_labels = val_labels.to_list()
test_texts = test_texts.to_list()
test_labels = test_labels.to_list()

In [43]:
train_encodings = tokenizer(train_texts, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, padding=True, truncation=True)
test_encodings = tokenizer(test_texts, padding=True, truncation=True)

We then create a tf dataset using the encodings and the labels.

In [44]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

## Fine-tuning the model

Now let us fine-tune our pre-trained model by training it with our custom dataset.  

We first instantiate a DistilBert config object, and customise it to suit our needs. In our case, we will just specify the *num_labels* to tell the model how many labels to use in the last layer (classification layer). You only need to specify this if you are doing multi-class classification. 

In [45]:
config = AutoConfig.from_pretrained("distilbert-base-uncased", 
                                    num_labels=2)

[INFO|configuration_utils.py:517] 2021-06-16 06:51:02,413 >> loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
[INFO|configuration_utils.py:553] 2021-06-16 06:51:02,416 >> Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.6.1",
  "vocab_size": 30522
}



We then instantiate a DistilBert model using this config object. If the config object is not passed, the default is a binary classification. The model is a a `tf.keras.Model` subclass. So you can train the model using Keras API such as `fit()`, or use Tensorflow custom training loops if you want to have more control over the training. The transformer library however, provides a Trainer class which abstract away the complex training loop, and supports distributed training on multi-GPU system. We will use this to train our model.

To use the Trainer class, we need to setup the training arguments such as number of epochs, batch sizes, warming up steps (commonly used in training Transformer model), weight decay (used to by Adam Optimizer for regularization purpose), learning rate, etc.

In [46]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10
)

[INFO|training_args.py:710] 2021-06-16 06:51:03,190 >> PyTorch: setting up devices
[INFO|training_args.py:616] 2021-06-16 06:51:03,193 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [47]:
## for distributed training on multi-gpu system, uncomment the following 

with training_args.strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",
        config=config)


[INFO|training_args_tf.py:192] 2021-06-16 06:51:04,855 >> Tensorflow: setting up strategy
[INFO|modeling_tf_utils.py:1261] 2021-06-16 06:51:04,967 >> loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers/fa107dc22c014df078a1b75235144a927f7e9764916222711f239b7ee6092ec9.bc4b731be56d8422e12b1d5bfa86fbd81d18d2770da1f5ac4f33640a17b7dde9.h5
[WARNING|modeling_tf_utils.py:1311] 2021-06-16 06:51:05,484 >> Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initial

We then define a function `compute_metrics()`  that will be used to compute metrics at evaluation. it takes in a EvalPrediction and return a dictionary string to metric values. In our case we just return the accuracy. 

In [48]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"acc": (preds == p.label_ids).mean()}

In [49]:
# We define a tensorboard writer 
writer = tf.summary.create_file_writer("tblogs")

trainer = TFTrainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    compute_metrics = compute_metrics,
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    tb_writer=writer
)

[INFO|trainer_tf.py:117] 2021-06-16 06:51:08,594 >> You are instantiating a Trainer but W&B is not installed. To use wandb logging, run `pip install wandb; wandb login` see https://docs.wandb.com/huggingface.
[INFO|trainer_tf.py:125] 2021-06-16 06:51:08,599 >> To use comet_ml logging, run `pip/conda install comet_ml` see https://www.comet.ml/docs/python-sdk/huggingface/


We start the training, and do the evaluation. On a single-GPU system, the training will around 6-7 minutes to complete. 

In [50]:
trainer.train()


[INFO|trainer_tf.py:528] 2021-06-16 06:51:12,791 >> ***** Running training *****
[INFO|trainer_tf.py:529] 2021-06-16 06:51:12,792 >>   Num examples = 2000
[INFO|trainer_tf.py:531] 2021-06-16 06:51:12,794 >>   Num Epochs = 1.0
[INFO|trainer_tf.py:532] 2021-06-16 06:51:12,795 >>   Instantaneous batch size per device = 16
[INFO|trainer_tf.py:534] 2021-06-16 06:51:12,796 >>   Total train batch size (w. parallel, distributed & accumulation) = 16
[INFO|trainer_tf.py:536] 2021-06-16 06:51:12,797 >>   Gradient Accumulation steps = 1
[INFO|trainer_tf.py:537] 2021-06-16 06:51:12,804 >>   Steps per epoch = 125
[INFO|trainer_tf.py:538] 2021-06-16 06:51:12,805 >>   Total optimization steps = 125


[INFO|trainer_tf.py:404] 2021-06-16 06:51:27,354 >> {'loss': 0.68808204, 'learning_rate': 1e-06, 'epoch': 0.08, 'step': 10}
[INFO|trainer_tf.py:404] 2021-06-16 06:51:35,859 >> {'loss': 0.6913229, 'learning_rate': 2e-06, 'epoch': 0.16, 'step': 20}
[INFO|trainer_tf.py:404] 2021-06-16 06:51:44,398 >> {'loss': 0.6924981, 'learning_rate': 2.9999999e-06, 'epoch': 0.24, 'step': 30}
[INFO|trainer_tf.py:404] 2021-06-16 06:51:52,978 >> {'loss': 0.69271827, 'learning_rate': 4e-06, 'epoch': 0.32, 'step': 40}
[INFO|trainer_tf.py:404] 2021-06-16 06:52:01,629 >> {'loss': 0.6924953, 'learning_rate': 5.0000003e-06, 'epoch': 0.4, 'step': 50}
[INFO|trainer_tf.py:404] 2021-06-16 06:52:10,309 >> {'loss': 0.69149435, 'learning_rate': 5.9999998e-06, 'epoch': 0.48, 'step': 60}
[INFO|trainer_tf.py:404] 2021-06-16 06:52:19,022 >> {'loss': 0.69050235, 'learning_rate': 6.9999996e-06, 'epoch': 0.56, 'step': 70}
[INFO|trainer_tf.py:404] 2021-06-16 06:52:27,795 >> {'loss': 0.68801606, 'learning_rate': 8e-06, 'epoch'

In [51]:
trainer.evaluate()

[INFO|trainer_tf.py:306] 2021-06-16 06:53:07,609 >> ***** Running Evaluation *****
[INFO|trainer_tf.py:307] 2021-06-16 06:53:07,613 >>   Num examples in dataset = 500
[INFO|trainer_tf.py:309] 2021-06-16 06:53:07,614 >>   Num examples in used in evaluation = 512
[INFO|trainer_tf.py:310] 2021-06-16 06:53:07,616 >>   Batch size = 32


[INFO|trainer_tf.py:404] 2021-06-16 06:53:18,216 >> {'eval_loss': 0.31664034724235535, 'eval_acc': 0.880859375, 'epoch': 1.0, 'step': 125}


{'eval_acc': 0.880859375, 'eval_loss': 0.31664034724235535}

Let's see how it performs on our test set. 

In [52]:
preds = trainer.predict(test_dataset)

[INFO|trainer_tf.py:306] 2021-06-16 06:53:18,239 >> ***** Running Prediction *****
[INFO|trainer_tf.py:307] 2021-06-16 06:53:18,243 >>   Num examples in dataset = 200
[INFO|trainer_tf.py:310] 2021-06-16 06:53:18,245 >>   Batch size = 32


The output from predict is logits, so we need to use a softmax to turn the values to probabilities and then use np.argmax to select the label with largest probalities.

In [53]:
tf_predictions = tf.nn.softmax(preds.predictions, axis=-1)

In [54]:
y_preds = np.argmax(tf_predictions, axis=-1)

In [55]:
from sklearn.metrics import classification_report

print(classification_report(preds.label_ids, y_preds))

              precision    recall  f1-score   support

           0       0.87      0.93      0.90       107
           1       0.91      0.84      0.87        93

    accuracy                           0.89       200
   macro avg       0.89      0.88      0.88       200
weighted avg       0.89      0.89      0.88       200



In [ ]:
#model.save_pretrained('./save_model/')

## Try out the model

Now let's try out our model with our own sentence. 

In [64]:
test_sentence = "I don't see how people can sit through this hour-long movie!"
#test_sentence = "The movie, though flawed, is still interesting enough."
test_sentence = 'The movie kept we on my toe all the time.'
inputs = tokenizer(test_sentence, return_tensors="tf")
out = model(inputs)
print(out)

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.06228681,  0.01608959]], dtype=float32)>, hidden_states=None, attentions=None)


In [65]:
print(np.argmax(tf.nn.softmax(out.logits, axis=-1)))

1


**Exercise:**

- Try to use BERT base-cased pretrained model and see if you get better or worse performance.
- Try to use BERT base-uncased pretrained model and see if you get better or worse performance.
- Try using a larger number of training samples. 
- Try multi-class classification using the this [dataset](https://sdaai-bucket.s3-ap-southeast-1.amazonaws.com/datasets/news.csv) that groups news title into 4 categories: e (entertainment), b (business), t (tech), m (medical/health). Original dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/News+Aggregator)